<a href="https://colab.research.google.com/github/shanxS/MLPlayground/blob/main/gpt_dev_0_01_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(text[:1000])

--2023-05-09 04:23:52--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2023-05-09 04:23:52 (114 MB/s) - ‘input.txt’ saved [1115394/1115394]

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be

In [22]:
chars = sorted(list(set(''.join(text))))
vocab_size = len(chars)

In [23]:
''.join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [34]:
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

stoi

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [25]:
import torch
data = torch.tensor(encode(text))

In [26]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [27]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x,y)
for i in range(block_size):
  context = x[:i+1]
  output = y[i]
  print(f'{context = } {output = } ')

tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor([47, 56, 57, 58,  1, 15, 47, 58])
context = tensor([18]) output = tensor(47) 
context = tensor([18, 47]) output = tensor(56) 
context = tensor([18, 47, 56]) output = tensor(57) 
context = tensor([18, 47, 56, 57]) output = tensor(58) 
context = tensor([18, 47, 56, 57, 58]) output = tensor(1) 
context = tensor([18, 47, 56, 57, 58,  1]) output = tensor(15) 
context = tensor([18, 47, 56, 57, 58,  1, 15]) output = tensor(47) 
context = tensor([18, 47, 56, 57, 58,  1, 15, 47]) output = tensor(58) 


In [28]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack( [data[i: i+block_size] for i in ix] )
  y = torch.stack( [data[i+1:i+block_size+1] for i in ix] )
  return x, y

xb, yb = get_batch("train")
print(xb.shape, yb.shape)
print(f'{xb=} \n {yb=}')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f'{context = }  {target = }')

torch.Size([4, 8]) torch.Size([4, 8])
xb=tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) 
 yb=tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
context = tensor([24])  target = tensor(43)
context = tensor([24, 43])  target = tensor(58)
context = tensor([24, 43, 58])  target = tensor(5)
context = tensor([24, 43, 58,  5])  target = tensor(57)
context = tensor([24, 43, 58,  5, 57])  target = tensor(1)
context = tensor([24, 43, 58,  5, 57,  1])  target = tensor(46)
context = tensor([24, 43, 58,  5, 57,  1, 46])  target = tensor(43)
context = tensor([24, 43, 58,  5, 57,  1, 46, 43])  target = tensor(39)
context = tensor([44])  target = tensor(53)
context = tensor([44, 53])  target = tensor(56)
context = tensor([44, 53, 56])  target = tensor(1)
context = t

In [29]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLangugageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    '''
    1st argument is size of dict
    2nd argument is size of embedding for each input token
    So the dim is vocab_size x C, where C is Channel
    '''
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, input, target = None):
    '''
    This is the output. Literally just dot product of input (B x T) and embedding table (vocab_size x channel) to get embedding
    I think it internally converts input into one hot encoding of size (B x T x vocab_size)
    B: number of inputs
    T: size of each input
    C: channel
    '''
    # print(input)
    # print(target)
    
    logits = self.token_embedding_table(input) # (B x T x C)

    if target is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(B*T)
      loss = F.cross_entropy(logits, target)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx)
      logits = logits[:, -1, :] # I dont understand why last index is predicted token
      logits = logits.exp()
      prob = logits / logits.sum(1, keepdims = True)
      idx_next = torch.multinomial(prob, num_samples=1, replacement=True)
      idx = torch.cat((idx, idx_next), dim =1)
    return idx


m = BigramLangugageModel(vocab_size)
# logits, loss = m(xb, yb)

In [36]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long), max_new_tokens = 100)[0].tolist()))


NRo ls, theresseys
Plorseelapinghiend n yof GLUCEN t l-h!
O:
AROLLUShers f dere! e!
QUCotouciullle's


In [37]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [38]:
batch_size = 32
for steps in range(10000):
  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  loss.backward()
  optimizer.step()

print(loss)

tensor(2.6665, grad_fn=<NllLossBackward0>)


In [35]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long), max_new_tokens = 400)[0].tolist()))


I oteng h hasbe pave pirance
RDe hicomyonthar's
PES:
ARod ith heno hatucenonthioneir thondy, y heltieiengerofuresssit ey
KIN d pe wither vouprrouthers nowl t,
K:

My hind thid pig t ouchos tes; st yo hind wotin grotonear 'so it t jod weancothanan hay. t le n frids, r loncave w hollular s O:
HIs; ht anje caike ineent.

Lavinde.
athave l.
KEONGBUThandspo be y,-hedarwnod y scar t aridesar, fre'shenou
